In [1]:
import pandas as pd
import numpy as np
import os
import numpy as np

In [2]:
train = pd.read_csv('~/ds-hack/data/train_data.csv')
train_complaints = pd.read_csv('~/ds-hack/data/complaints_train.csv')
test = pd.read_csv('~/ds-hack/data/test_data.csv')
sub = pd.read_csv('~/ds-hack/data/sample_submission.csv')

In [3]:
needed_columns = ['Минимальная глубина CPV (2017 год)', 'СуммаТендера', 'Организатор',
                 'Дата начала подачи предложений', 'Тип методу закупівлі', 'unique_id',
                 'Тип организатора', 'Статус лота', 'СтатусТендера', 'IDOrganizator',
                 'РегионОрганизатора', 'Регион поставки', 'Тендер',
                 'Дата начала периода уточнений', 'Дата завершения',
                 'Дата завершения подачи предложений', 'ШагУменьшенияЛот','СуммаГарантииПредложенияЛота']

In [4]:
train_agg = train[needed_columns]
test_agg = test[needed_columns]

### Filtering columns

In [5]:
train_target = train_agg.groupby('unique_id', as_index=False).count()[['unique_id', 'СуммаТендера']].rename({'СуммаТендера': 'target'}, axis=1).set_index('unique_id')
test_target = test_agg.groupby('unique_id', as_index=False).count()[['unique_id', 'СуммаТендера']].rename({'СуммаТендера': 'target'}, axis=1).set_index('unique_id')

### Replacing index

In [6]:
train_agg.index = train_agg.unique_id
train_agg.drop('unique_id', axis=1, inplace=True)
test.drop('unique_id', axis=1, inplace=True)

/home/alexanderonbysh/ds-hack/.env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
train_agg = train_agg.join(train_target)
test_agg = test_agg.join(test_target)
test_agg.index = test_agg.unique_id
test_agg.drop('unique_id', axis=1, inplace=True)
test_agg.drop('target', axis=1, inplace=True)

In [9]:
train_agg['Минимальная глубина CPV (2017 год)'] = train_agg['Минимальная глубина CPV (2017 год)'].replace('-', '000000000')
test_agg['Минимальная глубина CPV (2017 год)'] = test_agg['Минимальная глубина CPV (2017 год)'].replace('-', '000000000')

In [10]:
train_agg['Минимальная глубина CPV (2017 год)'] = train_agg['Минимальная глубина CPV (2017 год)'].apply(lambda x: x[:8])
test_agg['Минимальная глубина CPV (2017 год)'] = test_agg['Минимальная глубина CPV (2017 год)'].apply(lambda x: x[:8])

In [11]:
train_agg = train_agg[train_agg['Дата начала периода уточнений'] != '-']

In [12]:
train_agg['Дата завершения подачи предложений'] = pd.to_datetime(train_agg['Дата завершения подачи предложений'])
train_agg['Дата завершения'] = pd.to_datetime(train_agg['Дата завершения'])
train_agg['Дата начала периода уточнений'] = pd.to_datetime(train_agg['Дата начала периода уточнений'])
train_agg['Дата начала подачи предложений'] = pd.to_datetime(train_agg['Дата начала подачи предложений'])

test_agg['Дата завершения подачи предложений'] = pd.to_datetime(test_agg['Дата завершения подачи предложений'])
test_agg['Дата завершения'] = pd.to_datetime(test_agg['Дата завершения'])
test_agg['Дата начала периода уточнений'] = pd.to_datetime(test_agg['Дата начала периода уточнений'])
test_agg['Дата начала подачи предложений'] = pd.to_datetime(test_agg['Дата начала подачи предложений'])

In [13]:
train_agg.loc[train_agg['СуммаГарантииПредложенияЛота'] == '-', 'СуммаГарантииПредложенияЛота'] = 0
test_agg.loc[test_agg['СуммаГарантииПредложенияЛота'] == '-', 'СуммаГарантииПредложенияЛота'] = 0

In [14]:
train_agg['cpv_group'] = train_agg['Минимальная глубина CPV (2017 год)'].str[:4]
test_agg['cpv_group'] = test_agg['Минимальная глубина CPV (2017 год)'].str[:4]

In [16]:
train_agg['time_diff'] = (train_agg['Дата завершения'].values.astype(np.float64) - train_agg['Дата начала подачи предложений'].values.astype(np.float64)) / 1e15
test_agg['time_diff'] = (test_agg['Дата завершения'].values.astype(np.float64) - test_agg['Дата начала подачи предложений'].values.astype(np.float64)) / 1e15

In [17]:
train_agg = train_agg[train_agg['time_diff'] > 0]
test_agg = test_agg[test_agg['time_diff'] > 0]

### Saving datasets

In [18]:
train_agg.to_csv('/home/alexanderonbysh/ds-hack/data/train_agg_4.csv')

In [19]:
test_agg.to_csv('/home/alexanderonbysh/ds-hack/data/test_agg_4.csv')